In [1]:
using JuMP, Ipopt, SparseArrays, LinearAlgebra, NamedArrays, DelimitedFiles, PyPlot, CSV

# Include necessary functions
include("Functions/ybus_matrix.jl");        # Function to calculate Ybus matrix (G,B)
include("Functions/variables_polar.jl");    # Function to define variables
include("Functions/constraints_polar.jl");  # Function to define constraints
include("Functions/cost_polar.jl");         # Function to define cost
include("Functions/misc_polar.jl");         # Function to define start point,map input,print output,plot
include("Functions/parse_gld_day1.jl");      # Function to parse GridLabD system data
include("Functions/fixed_point.jl");      # Function to parse GridLabD system data


case_name = "case13_glm"
case_name = "GC-12.47-1"
#case_name = "R1-12.47-3"
#case_name = "R1-25.00-1"
#case_name = "R1-12.47-1"
#case_name = "turkey_solar"
data_file = open(string("data/",case_name,"/data.xml"))
data = readlines(data_file)
z_file = open(string("data/",case_name,"/z_dump.xml"))
z_dump = readlines(z_file);
house_sch = CSV.read(string("House_",case_name,"_profile.csv"),header=false,type=Float64)
    global house_sch = convert(Matrix,house_sch);
    PV_sch = CSV.read(string("PV_",case_name,"_profile.csv"),header=false,type=Float64)
    global PV_sch = convert(Matrix,PV_sch);
    idx_PV = CSV.read(string("DSPV_list.csv"),header=false,type=Float64)
    global idx_PV = Int.(convert(Matrix,idx_PV).+3);

    # Other inputs
    crit_node = "all"
    init_op = 1   # 1- Flat start, 2- GridlabD point, 3-Specified point
    global PV_rat = 0.025;

    # Read all system data and calculate number of buses, generators, loads
    global count_sch = 1
    parse_gld();
    par_nam = ["nb"; "ngy"; "nly"; "nld"; "nb_nph"; "ngy_nph"; "nly_nph"; "nld_nph"]
    par_val = [nb ngy nly nld nb_nph ngy_nph nly_nph nld_nph];
    par_det = NamedArray( par_val', (par_nam, [:Value]), ("Par","Val"))
    @show par_det
    print("-------------------\n")

ArgumentError: ArgumentError: "House_GC-12.47-1_profile.csv" is not a valid file

In [3]:
Cm = spzeros(nb);
for i=1:nb
    if !isempty(findall(idx->idx==i,branch[:,1:2])) 
        Cm[i]=1      
    end
end
sum(Cm) == nb

true

In [3]:
idx_node_f = findall(idx -> idx == "\t\t\t<node>", data); idx_node_t = findall(idx -> idx == "\t\t\t</node>", data)
idx_load_f = findall(idx -> idx == "\t\t\t<load>", data); idx_load_t = findall(idx -> idx == "\t\t\t</load>", data)
idx_tnode_f = findall(idx -> idx == "\t\t\t<triplex_node>", data); idx_tnode_t = findall(idx -> idx == "\t\t\t</triplex_node>", data)
idx_meter_f = findall(idx -> idx == "\t\t\t<meter>", data); idx_meter_t = findall(idx -> idx == "\t\t\t</meter>", data)
idx_tmeter_f = findall(idx -> idx == "\t\t\t<triplex_meter>", data); idx_tmeter_t = findall(idx -> idx == "\t\t\t</triplex_meter>", data)
idx_bus_f = [idx_node_f; idx_load_f; idx_tnode_f]; idx_bus_t = [idx_node_t; idx_load_t; idx_tnode_t]
x=get_data(data,nb,idx_bus_f,idx_bus_t,"<busflags>",3)
xx=findall(idx -> idx != "HASSOURCE" ,x)
length(xx)==0

true

In [4]:
vl = findall(idx -> idx < 0.9 && idx > 0, bus[:,9:2:13])
vu = findall(idx -> idx > 1.1 && idx > 0, bus[:,9:2:13])
length(vl)+length(vu)==0

true

In [13]:
Pl_tot = tot_load/baseMVA*1e3
#Pg_tot = sum(ygen[:,2])*1e3
#Pg_tot - Pl_tot


62.1651

In [9]:
soln_VU = readdlm(string("VPQ_",case_name,"_soln.csv"), ',')
V1 = soln_VU[:,1]; V1 = soln_VU[:,1]; θ1 = soln_VU[:,2];
fs = 10; xs = 5; ys = 3
Vm_a = zeros(nb,1);Vm_b = zeros(nb,1);Vm_c = zeros(nb,1)
θ_a = zeros(nb,1);θ_b = zeros(nb,1);θ_c = zeros(nb,1)
t = range(1,stop=nb)

for i=1:nb
    for j=Int.(sum(bus_count[1:i-1])+1:sum(bus_count[1:i]))    
        if bus_ϕ[j] == "A" 
            Vm_a[i] = V1[j]
            θ_a[i] = θ1[j]    
        elseif bus_ϕ[j] == "B"  
            Vm_b[i] = V1[j]
            θ_b[i] = θ1[j]
        else
            Vm_c[i] = V1[j]
            θ_c[i] = θ1[j] 
        end
    end
end
for i=1:nb
    if abs(θ_b[i]) >= 1e-5
        θ_b[i] = abs(θ_b[i] - θ_a[i] + 120)
    end
    if abs(θ_c[i]) >= 1e-5
        θ_c[i] = abs(θ_c[i] - θ_a[i] - 120)
    end        
    θ_a[i] = 0             
end
[Vm_a θ_a Vm_b θ_b Vm_c θ_c]


15×6 Array{Float64,2}:
 1.0207  0.0  1.0129  0.6179  1.0105  0.49  
 1.0624  0.0  1.0624  0.0     1.0624  0.0006
 1.0211  0.0  1.0164  0.3973  1.012   0.2497
 1.0     0.0  1.0     0.0     1.0     0.0   
 0.977   0.0  1.0234  1.4638  0.9679  1.8466
 0.9732  0.0  0.0     0.0     0.9689  1.9985
 1.0009  0.0  0.994   0.8336  0.9916  0.7079
 0.0     0.0  0.9957  1.9509  1.0081  1.6378
 0.0     0.0  0.9936  2.0306  1.0035  1.5514
 0.9641  0.0  0.0     0.0     0.9725  2.4461
 0.9764  0.0  1.0229  1.3342  0.9686  1.8165
 0.9692  0.0  1.0238  1.3959  0.965   2.0238
 0.9764  0.0  1.0228  1.3333  0.9686  1.8163
 0.9734  0.0  0.0     0.0     0.9679  2.0126
 1.0101  0.0  1.0175  0.0108  1.0004  0.2007

In [9]:
count =0
for i=1:3, j=1:3
    count = count +1
end
count

9